- Update: 2022.12.27

# 0. 配置参数

- `dict_db`：数据库文件的相关信息
    - `file_name`：文件的路径
    - `sheet_name`：工作表名称
    - `head`：表头行数
    - `col_sku`：SKU号所在列的列名
    - `col_oe`：OE号所在列的列名
- `dict_table`：需要数据库查重的文件的相关信息
    - `file_name`：文件的路径
    - `sheet_name`：工作表名称
    - `head`：表头行数
    - `col_oe`：OE号所在列的列名
- `out`：保存结果的路径

In [1]:
dict_db = {'file_name': '../../file/【询价单】玉环博宇卡钳-20221026-工厂10.31回复.xls',
           'sheet_name': 'PTsku',
           'head': 1,
           'col_sku': 'SKU',
           'col_oe': 'OE'}

dict_table = {'file_name': '../../file/【询价单】玉环博宇卡钳-20221026-工厂10.31回复.xls',
              'sheet_name': '目录',
              'head': 4,
              'col_oe': 'OE'}

out = '../../file/数据库查重_NOW().xlsx'

In [2]:
import re

In [3]:
def normalise_oe(oe, test=False):
    oe = str(oe).strip()
    
    if test:
        print(f'oe:\n{oe}')
    
    oe = oe.replace('\u200e', '')
    oe = oe.replace('\n', ';')
    
    oe = re.sub(r'(GM|Subaru|Dodge|Ford)[:：]', '', oe)
    
    if test:
        print()
        print(f'oe:\n{oe}')
    
    oe = re.sub(r'[\(（].*?[\)）]', '', oe)
    
    if test:
        print()
        print(f'oe:\n{oe}')
    
    oe = re.sub(r'[\u4e00-\u9fa5]', '', oe)
    
    if test:
        print()
        print(f'oe:\n{oe}')
    
    oe = oe.replace('-', '')
    oe = re.sub(r'\*+[2-9]?', '', oe)
    oe = oe.replace(' ', ';')
    oe = oe.replace('.', ';')
    oe = oe.replace('//', ';')
    oe = re.sub(r'[,，。;；+=/、]', ';', oe)
    
    oe = oe.replace(';;;', ';')
    oe = oe.replace(';;', ';')
    
    if test:
        print()
        print(f'oe:\n{oe}')
    
    return oe.upper()

# 1. 处理数据库

In [4]:
import pandas as pd

In [5]:
df_db = pd.read_excel(dict_db['file_name'], header=dict_db['head']-1, sheet_name=dict_db['sheet_name'])

df_db

,SKU,OE
0,BC000040,343009;SEG000040
1,BC000050,343008;SEG000050
2,BC049108,343242;7701049108
3,BC049109,343243;7701049109
4,BC049GF0,343976;1460305;1701884;6G912D049GC;1465777;173...
...,...,...
5133,BCN43274SLR,343274;343275;522311;522312;542475;542476;9317...
5134,BCN44150JLR,344150;344151;4401.Q0;4401.Q1;530081;530082
5135,BCN43316SLR,44011BU000;721331;343316;44001BM400;44001BU000...
5136,EBCN44662SLR,3C0615403C;3C0615403D;530111;19-3784;344662;3C...


In [6]:
for i in range(len(df_db)):
    df_db.loc[i, dict_db['col_oe']] = normalise_oe(df_db.loc[i, dict_db['col_oe']])

df_db

,SKU,OE
0,BC000040,343009;SEG000040
1,BC000050,343008;SEG000050
2,BC049108,343242;7701049108
3,BC049109,343243;7701049109
4,BC049GF0,343976;1460305;1701884;6G912D049GC;1465777;173...
...,...,...
5133,BCN43274SLR,343274;343275;522311;522312;542475;542476;9317...
5134,BCN44150JLR,344150;344151;4401;Q0;4401;Q1;530081;530082
5135,BCN43316SLR,44011BU000;721331;343316;44001BM400;44001BU000...
5136,EBCN44662SLR,3C0615403C;3C0615403D;530111;193784;344662;3C0...


# 2. 处理询价单

In [7]:
df_table = pd.read_excel(dict_table['file_name'], header=dict_table['head']-1, sheet_name=dict_table['sheet_name'])

df_table

,Ref No,OE,Application,NOTE,Axle,已开发SKU
0,19-2766,4773060130/ 47730-60261,Toyota 4 Runner 2009 4.7L,NaN,FRONT,NaN
1,19-2767,4775060130 / 47750-60261,Toyota 4 Runner 2009 4.7L,NaN,FRONT,NaN
2,19-B2848,581802DA10,Hyundai Elantra 2005 2.0L,NaN,FRONT,NaN
3,19-B2849,581902DA10,Hyundai Elantra 2005 2.0L,NaN,FRONT,NaN
4,19-B2653,58311 3AA00,Hyundai Santa Fe 2006 2.4L,NaN,REAR,NaN
...,...,...,...,...,...,...
539,19-2984,4773004090,Toyota Tacoma 2016 2.7L,NaN,FRONT,NaN
540,19-B2950,4775021030,Scion Tc 2010 2.4L,NaN,REAR,NaN
541,19-B2951,4773021030,Scion Tc 2010 2.4L,NaN,REAR,NaN
542,19-B6274,477500E030 / 477500E031,Toyota Highlander 2018 3.5L,NaN,FRONT,NaN


In [8]:
for i in range(len(df_table)):
    df_table.loc[i, dict_table['col_oe']] = normalise_oe(df_table.loc[i, dict_table['col_oe']])

df_table

,Ref No,OE,Application,NOTE,Axle,已开发SKU
0,19-2766,4773060130;4773060261,Toyota 4 Runner 2009 4.7L,NaN,FRONT,NaN
1,19-2767,4775060130;4775060261,Toyota 4 Runner 2009 4.7L,NaN,FRONT,NaN
2,19-B2848,581802DA10,Hyundai Elantra 2005 2.0L,NaN,FRONT,NaN
3,19-B2849,581902DA10,Hyundai Elantra 2005 2.0L,NaN,FRONT,NaN
4,19-B2653,58311;3AA00,Hyundai Santa Fe 2006 2.4L,NaN,REAR,NaN
...,...,...,...,...,...,...
539,19-2984,4773004090,Toyota Tacoma 2016 2.7L,NaN,FRONT,NaN
540,19-B2950,4775021030,Scion Tc 2010 2.4L,NaN,REAR,NaN
541,19-B2951,4773021030,Scion Tc 2010 2.4L,NaN,REAR,NaN
542,19-B6274,477500E030;477500E031,Toyota Highlander 2018 3.5L,NaN,FRONT,NaN


# 3. 查重

In [9]:
list_chachong = []
for i in range(len(df_table)):
    print(i)
    table_oes = df_table.loc[i, dict_table['col_oe']].split(';')
    
    list_sku = []
    for j in range(len(df_db)):
        db_oes = df_db.loc[j, dict_db['col_oe']].split(';')
        
        for k in range(len(table_oes)):
            if table_oes[k] in db_oes:
                list_sku.append(df_db.loc[j, dict_db['col_sku']])
                break
    
    set_sku = set(list_sku)
    if len(set_sku) == 0:
        list_chachong.append('')
    else:
        set_sku = [str(i) for i in set_sku]
        list_chachong.append(';'.join(set_sku))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [10]:
df_table['查重'] = list_chachong

df_table

,Ref No,OE,Application,NOTE,Axle,已开发SKU,查重
0,19-2766,4773060130;4773060261,Toyota 4 Runner 2009 4.7L,NaN,FRONT,NaN,BC477363J;BCB92766;BC477363;BC477563JLR;BC4775...
1,19-2767,4775060130;4775060261,Toyota 4 Runner 2009 4.7L,NaN,FRONT,NaN,BC477563J;BC477563;BC477563JLR;BC477563LR;BCB7...
2,19-B2848,581802DA10,Hyundai Elantra 2005 2.0L,NaN,FRONT,NaN,BCB92848LR;BCB92848
3,19-B2849,581902DA10,Hyundai Elantra 2005 2.0L,NaN,FRONT,NaN,BCB92848LR;BCB92849
4,19-B2653,58311;3AA00,Hyundai Santa Fe 2006 2.4L,NaN,REAR,NaN,
...,...,...,...,...,...,...,...
539,19-2984,4773004090,Toyota Tacoma 2016 2.7L,NaN,FRONT,NaN,BCN92984J
540,19-B2950,4775021030,Scion Tc 2010 2.4L,NaN,REAR,NaN,BCB92950LR;BCB92950
541,19-B2951,4773021030,Scion Tc 2010 2.4L,NaN,REAR,NaN,BCB92950LR;BCB92951
542,19-B6274,477500E030;477500E031,Toyota Highlander 2018 3.5L,NaN,FRONT,NaN,BCB96274;BCB96274LR


# 4. 保存

In [11]:
import datetime

writer = pd.ExcelWriter(out.replace('NOW()', datetime.datetime.now().strftime("%Y%m%d_%H%M%S")))
df_table.to_excel(writer, index=False)
writer.close()